In [38]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split

from uni2ts.eval_util.plot import plot_single, plot_next_multi
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule

from neuralforecast.core import NeuralForecast
from neuralforecast.models import NHITS, NBEATSx, TSMixerx

from neuralforecast.losses.numpy import mae, mse

import warnings
warnings.filterwarnings('ignore')


In [39]:
# from pynvml.smi import nvidia_smi
# nvsmi = nvidia_smi.getInstance()
# nvsmi.DeviceQuery('memory.free, memory.total')

In [40]:
df = pd.read_csv(r"C:\Users\HP\Documents\AIISC\TimeSeries\Data\FF_resampled.csv")
# Step 1: Convert '_time' column to datetime format
df['_time'] = pd.to_datetime(df['_time'], format='ISO8601')

# # Step 2: Set '_time' as index
# df.set_index('_time', inplace=True)
df['unique_id'] = 0

# # Step 3: Sort by index to ensure chronological order
# df.sort_index(inplace=True)

In [41]:
df

,_time,Q_VFD1_Temperature,Q_VFD2_Temperature,Q_VFD3_Temperature,Q_VFD4_Temperature,Q_Cell_CycleCount,CycleState,I_R01_Gripper_Load,I_R01_Gripper_Pot,M_R01_BJointAngle_Degree,...,Cycle_Count_New,I_Stopper1_Status,I_Stopper2_Status,I_Stopper3_Status,I_Stopper4_Status,I_Stopper5_Status,I_MHS_GreenRocketTray,Description,actual_state,unique_id
0,2023-12-11 13:04:12+00:00,68.129944,68.129944,68.129944,68.002197,1.0,1.0,483.428571,11317.714286,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2023-12-11 13:04:13+00:00,68.129944,68.129944,68.129944,68.002197,1.0,1.0,510.400000,11327.500000,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2023-12-11 13:04:14+00:00,68.129944,68.118268,61.316949,68.002197,1.0,1.0,522.200000,11315.800000,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2023-12-11 13:04:15+00:00,68.129944,68.129944,68.129944,68.002197,1.0,1.0,513.200000,11351.500000,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2023-12-11 13:04:16+00:00,68.129944,68.129944,68.129944,68.002197,1.0,1.0,515.700000,11329.700000,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104606,2023-12-12 18:07:38+00:00,90.130188,84.130272,89.647152,80.974072,275.0,20.0,426.500000,11172.700000,-90.001137,...,292.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0
104607,2023-12-12 18:07:39+00:00,90.130188,84.130272,89.630272,89.938599,275.0,20.0,379.700000,11226.600000,-90.001137,...,292.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0
104608,2023-12-12 18:07:40+00:00,90.130188,84.130272,89.630272,89.908936,275.0,20.0,400.200000,11190.000000,-90.001137,...,292.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0
104609,2023-12-12 18:07:41+00:00,90.130188,84.130272,89.630272,89.886963,275.0,20.4,398.000000,11182.200000,-90.001137,...,292.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0


In [42]:
df.columns

Index(['_time', 'Q_VFD1_Temperature', 'Q_VFD2_Temperature',
       'Q_VFD3_Temperature', 'Q_VFD4_Temperature', 'Q_Cell_CycleCount',
       'CycleState', 'I_R01_Gripper_Load', 'I_R01_Gripper_Pot',
       'M_R01_BJointAngle_Degree', 'M_R01_LJointAngle_Degree',
       'M_R01_RJointAngle_Degree', 'M_R01_SJointAngle_Degree',
       'M_R01_TJointAngle_Degree', 'M_R01_UJointAngle_Degree',
       'I_R02_Gripper_Load', 'I_R02_Gripper_Pot', 'M_R02_BJointAngle_Degree',
       'M_R02_LJointAngle_Degree', 'M_R02_RJointAngle_Degree',
       'M_R02_SJointAngle_Degree', 'M_R02_TJointAngle_Degree',
       'M_R02_UJointAngle_Degree', 'I_R03_Gripper_Load', 'I_R03_Gripper_Pot',
       'M_R03_BJointAngle_Degree', 'M_R03_LJointAngle_Degree',
       'M_R03_RJointAngle_Degree', 'M_R03_SJointAngle_Degree',
       'M_R03_TJointAngle_Degree', 'M_R03_UJointAngle_Degree',
       'I_R04_Gripper_Load', 'I_R04_Gripper_Pot', 'M_R04_BJointAngle_Degree',
       'M_R04_LJointAngle_Degree', 'M_R04_RJointAngle_Degree',
   

In [43]:
moirai_df = df.set_index('_time')

ds = PandasDataset.from_long_dataframe(
    moirai_df,
    target='Description',
    item_id='unique_id',
    feat_dynamic_real=["Q_VFD1_Temperature", 
                       "Q_VFD2_Temperature", 
                       "Q_VFD3_Temperature", 
                       "Q_VFD4_Temperature", 
                       "Q_Cell_CycleCount", 
                       "CycleState", 
                       "I_R01_Gripper_Load", 
                       "I_R01_Gripper_Pot", 
                       "M_R01_BJointAngle_Degree", 
                       "M_R01_LJointAngle_Degree", 
                       "M_R01_RJointAngle_Degree", 
                       "M_R01_SJointAngle_Degree", 
                       "M_R01_TJointAngle_Degree", 
                       "M_R01_UJointAngle_Degree", 
                       "I_R02_Gripper_Load", 
                       "I_R02_Gripper_Pot", 
                       "M_R02_BJointAngle_Degree", 
                       "M_R02_LJointAngle_Degree", 
                       "M_R02_RJointAngle_Degree", 
                       "M_R02_SJointAngle_Degree", 
                       "M_R02_TJointAngle_Degree", 
                       "M_R02_UJointAngle_Degree", 
                       "I_R03_Gripper_Load", 
                       "I_R03_Gripper_Pot", 
                       "M_R03_BJointAngle_Degree", 
                       "M_R03_LJointAngle_Degree", 
                       "M_R03_RJointAngle_Degree", 
                       "M_R03_SJointAngle_Degree", 
                       "M_R03_TJointAngle_Degree", 
                       "M_R03_UJointAngle_Degree", 
                       "I_R04_Gripper_Load", 
                       "I_R04_Gripper_Pot", 
                       "M_R04_BJointAngle_Degree", 
                       "M_R04_LJointAngle_Degree", 
                       "M_R04_RJointAngle_Degree", 
                       "M_R04_SJointAngle_Degree", 
                       "M_R04_TJointAngle_Degree", 
                       "M_R04_UJointAngle_Degree", 
                       "Cycle_Count_New", 
                       "I_Stopper1_Status", 
                       "I_Stopper2_Status", 
                       "I_Stopper3_Status", 
                       "I_Stopper4_Status", 
                       "I_Stopper5_Status", 
                       "I_MHS_GreenRocketTray", 
                       "actual_state"
]
)


In [44]:
# test_size = 50
test_size = 100

horizon = 7

train, test_template = split(
    ds, offset=-test_size
)

test_data = test_template.generate_instances(
    prediction_length=horizon,
    windows=test_size//horizon,
    distance=horizon
)


In [45]:
model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-small"),
    prediction_length=horizon,
    context_length=200,
    patch_size="auto",
    num_samples=100,
    target_dim=1,
    feat_dynamic_real_dim=ds.num_feat_dynamic_real,
    past_feat_dynamic_real_dim=ds.num_past_feat_dynamic_real,
)


In [46]:
len(test_data.input)

14

In [47]:
# import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


In [48]:
predictor = model.create_predictor(batch_size=2)
forecasts = predictor.predict(test_data.input)
# forecasts = list(forecasts)

In [49]:
forecasts = list(forecasts)

KeyboardInterrupt: 

In [ ]:
forecasts

[gluonts.model.forecast.SampleForecast(info=None, item_id='0', samples=array([[  0.99999964,   0.99999976,   1.        ,   0.9999996 ,
           0.99999976,   0.9999996 ,   0.9999996 ],
        [  0.99999964,   0.9999999 ,   0.9999999 ,   0.99999946,
           0.99999976,   0.99999964,   0.9999997 ],
        [  0.99999946,   1.0013342 ,   1.0000005 ,   0.99999964,
           0.99999964,   0.9999997 ,   0.9999994 ],
        [  0.99999946,   0.9999997 ,   0.99999976,   0.9999995 ,
           0.99999994,   0.99999976,   0.9999997 ],
        [  0.9999995 ,   0.9999996 ,   0.9999996 ,   0.9999995 ,
           0.9999997 ,   0.9999998 ,   1.0000001 ],
        [  0.99999934,   0.99999964,   0.9999996 ,   0.99999934,
           0.99999976,   0.9999999 ,   1.0087658 ],
        [  0.9999995 ,   0.9999996 ,   0.9999999 ,   0.9999997 ,
           0.99999964,   0.9999997 ,   0.99999976],
        [  1.0000002 ,   0.9999995 ,   0.9999997 ,   0.99999964,
           0.9999995 ,   0.9999999 ,   0.99999

In [14]:
def get_median_and_ci(data, 
                      horizon,
                      id,
                      confidence=0.80):

    n_samples, n_timesteps = data.shape
    
    # Calculate the median for each timestep
    medians = np.median(data, axis=0)
    
    # Calculate the lower and upper percentile for the given confidence interval
    lower_percentile = (1 - confidence) / 2 * 100
    upper_percentile = (1 + confidence) / 2 * 100
    
    # Calculate the lower and upper bounds for each timestep
    lower_bounds = np.percentile(data, lower_percentile, axis=0)
    upper_bounds = np.percentile(data, upper_percentile, axis=0)

    # Create a DataFrame with the results
    df = pd.DataFrame({
        'unique_id': id,
        'Moirai': medians,
        f'Moirai-lo-{int(confidence*100)}': lower_bounds,
        f'Moirai-hi-{int(confidence*100)}': upper_bounds
    })
    
    return df


In [ ]:
moirai_preds = [
    get_median_and_ci(
        data=forecasts[i].samples,
        horizon=horizon,
        id=1
    )
    for i in range(len(forecasts))
]


In [22]:
moirai_preds_df = pd.concat(moirai_preds, axis=0, ignore_index=True)


In [23]:
moirai_preds_df

,unique_id,Moirai,Moirai-lo-80,Moirai-hi-80
0,1,1.000000,0.999999,1.0
1,1,1.000000,0.999999,1.0
2,1,1.000000,0.999999,1.0
3,1,1.000000,0.999999,1.0
4,1,1.000000,1.000000,1.0
...,...,...,...,...
93,1,0.999999,0.999999,1.0
94,1,0.999999,0.999999,1.0
95,1,1.000000,0.999999,1.0
96,1,1.000000,0.999999,1.0


In [24]:
xcols=["Q_VFD1_Temperature", 
                       "Q_VFD2_Temperature", 
                       "Q_VFD3_Temperature", 
                       "Q_VFD4_Temperature", 
                       "Q_Cell_CycleCount", 
                       "CycleState", 
                       "I_R01_Gripper_Load", 
                       "I_R01_Gripper_Pot", 
                       "M_R01_BJointAngle_Degree", 
                       "M_R01_LJointAngle_Degree", 
                       "M_R01_RJointAngle_Degree", 
                       "M_R01_SJointAngle_Degree", 
                       "M_R01_TJointAngle_Degree", 
                       "M_R01_UJointAngle_Degree", 
                       "I_R02_Gripper_Load", 
                       "I_R02_Gripper_Pot", 
                       "M_R02_BJointAngle_Degree", 
                       "M_R02_LJointAngle_Degree", 
                       "M_R02_RJointAngle_Degree", 
                       "M_R02_SJointAngle_Degree", 
                       "M_R02_TJointAngle_Degree", 
                       "M_R02_UJointAngle_Degree", 
                       "I_R03_Gripper_Load", 
                       "I_R03_Gripper_Pot", 
                       "M_R03_BJointAngle_Degree", 
                       "M_R03_LJointAngle_Degree", 
                       "M_R03_RJointAngle_Degree", 
                       "M_R03_SJointAngle_Degree", 
                       "M_R03_TJointAngle_Degree", 
                       "M_R03_UJointAngle_Degree", 
                       "I_R04_Gripper_Load", 
                       "I_R04_Gripper_Pot", 
                       "M_R04_BJointAngle_Degree", 
                       "M_R04_LJointAngle_Degree", 
                       "M_R04_RJointAngle_Degree", 
                       "M_R04_SJointAngle_Degree", 
                       "M_R04_TJointAngle_Degree", 
                       "M_R04_UJointAngle_Degree", 
                       "Cycle_Count_New", 
                       "I_Stopper1_Status", 
                       "I_Stopper2_Status", 
                       "I_Stopper3_Status", 
                       "I_Stopper4_Status", 
                       "I_Stopper5_Status", 
                       "I_MHS_GreenRocketTray", 
                       "actual_state"
]

In [25]:
test_size

100

In [26]:
len(moirai_preds_df)

98

In [27]:
all_preds = pd.DataFrame({
    'unique_id': [0] * len(moirai_preds_df),
    'Description': df['Description'].tail(len(moirai_preds_df)).values
})



all_preds['Moirai'] = moirai_preds_df['Moirai'].values

all_preds = all_preds.reset_index(drop=False)

In [28]:
from utilsforecast.losses import mae, smape, mse
from utilsforecast.evaluation import evaluate

evaluation = evaluate(
    all_preds,
    metrics=[mae, smape, mse],
    target_col='Description',
    id_col='unique_id'
)


In [29]:
evaluation

,unique_id,metric,index,Moirai
0,0,mae,47.520408,3.971616e-07
1,0,smape,0.914954,1.985808e-07
2,0,mse,3056.500000,1.646212e-13


In [31]:
type(evaluation)

pandas.core.frame.DataFrame

In [33]:
evaluation.to_csv(r"C:\Users\HP\Documents\AIISC\TimeSeries\Data\metrics.csv")

In [37]:
evaluation['Moirai'][0]

3.9716156161561305e-07